In [ ]:
# default_exp data.statistics

In [ ]:
# export

from typing import List, Any, Dict, Union, Optional
import os

import librosa
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from pydub.utils import mediainfo_json
import seaborn as sns
import speechmetrics
from wordcloud import WordCloud, STOPWORDS
from wordfreq import word_frequency

from uberduck_ml_dev.text.util import text_to_sequence

In [ ]:
# export


def word_frequencies(text: str, language: str = "en") -> List[float]:
    """
    Calculate the frequency [0-1] which the words appear in the english language
    """
    freqs = []
    for word in text.split():
        freqs.append(word_frequency(word, language))
    return freqs


def create_wordcloud(text: str, output_file: str):
    """
    Creates and saves a wordcloud in the shape of a duck to the designated output file.
    """
    mask = np.array(ImageOps.invert(Image.open("duck.png").convert("RGB")))
    wc = WordCloud(
        background_color="white",
        max_words=3000,
        mask=mask,
        stopwords=set(STOPWORDS),
        contour_width=7,
        contour_color="steelblue",
    )
    wc.generate(text)
    wc.to_file(output_file)


def count_frequency(arr: List[Any]) -> Dict[Any, int]:
    """
    Calculates the frequency that a value appears in a list
    """

    frequency_dict = {}
    for val in arr:
        if val in frequency_dict.keys():
            frequency_dict[val] += 1
        else:
            frequency_dict[val] = 1

    return {
        k: v
        for k, v in sorted(
            frequency_dict.items(), key=lambda item: item[1], reverse=True
        )
    }


def pace_character(
    text: str, audio: Union[str, np.ndarray], sr: Optional[int] = None
) -> float:
    """
    Calculates the number of characters in the text per second of the audio file. Audio can be a file path or an np array.
    """
    if isinstance(audio, str):
        audio, sr = librosa.load(audio, sr=None)
    else:
        assert sr, "Sampling rate must be provided if audio is np array"

    return len(text) / librosa.get_duration(audio, sr=sr)


def pace_phoneme(
    text: str, audio: Union[str, np.ndarray], sr: Optional[int] = None
) -> float:
    """
    Calculates the number of phonemes in the text per second of the audio. Audio can be a file path or an np array.
    """
    if isinstance(audio, str):
        audio, sr = librosa.load(audio, sr=None)
    else:
        assert sr, "Sampling rate must be provided if audio is np array"

    arpabet_seq = text_to_sequence(text, ["english_cleaners"], p_arpabet=1.0)
    return len(arpabet_seq) / librosa.get_duration(audio, sr=sr)


def get_sample_format(wav_file: str):
    """
    Get sample format of the .wav file: https://trac.ffmpeg.org/wiki/audio%20types
    """
    filename, file_extension = os.path.splitext(wav_file)
    assert file_extension == ".wav", ".wav file must be supplied"

    info = mediainfo_json(wav_file)
    audio_streams = [x for x in info["streams"] if x["codec_type"] == "audio"]
    return audio_streams[0].get("sample_fmt")


def absolute_metrics(wav_file: str, window_length: Optional[int]) -> Dict[str, float]:
    """
    Returns a Dict[str,float] with keys "mosnet" and "srmr"
    """
    filename, file_extension = os.path.splitext(wav_file)
    assert file_extension == ".wav", ".wav file must be supplied"

    metrics = speechmetrics.load("absolute", window_length)
    return metrics(wav_file)

In [ ]:
p = "/home/ubuntu/data/uberduck-multispeaker/bullwinkle/wavs/1.wav"
x, sr = librosa.load(p)
s = "Hello out there in TV land!"

assert pace_phoneme(text=s, audio=x, sr=22050) == pace_phoneme(
    text=s, audio=p, sr=22050
), "Results should be equal"
assert pace_character(text=s, audio=x, sr=22050) == pace_character(
    text=s, audio=p, sr=22050
), "Results should be equal"